In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# Bidirectional Breadth First Search

The function `search` takes three arguments to solve a *search problem*:
- `start` is the *start state* of the search problem,
- `goal` is the *goal state*, and
- `next_states` is a function with signature $\texttt{next_states}:Q \rightarrow 2^Q$, where $Q$ is the set of states.
   For every state $s \in Q$, $\texttt{next_states}(s)$ is the set of states that can be reached from $s$ in one step.

If successful, `search` returns a path from `start` to `goal` that is a solution of the search problem
$$ \langle Q, \texttt{next_states}, \texttt{start}, \texttt{goal} \rangle. $$
The implementation of `search` uses *bidirectional breadth first search* to find a path from `start` to `goal`.

In [2]:
def search(start, goal, next_states):        
    FrontierA = { start }
    ParentA   = { start: start }
    FrontierB = { goal }
    ParentB   = { goal: goal } 
    while FrontierA and FrontierB:
        NewFrontier = set()
        for s in FrontierA:
            for ns in next_states(s):
                if ns not in ParentA:
                    NewFrontier |= { ns }
                    ParentA[ns]  = s
                    if ns in ParentB:
                        return combinePaths(ns, ParentA, ParentB)
        FrontierA   = NewFrontier
        NewFrontier = set()
        for s in FrontierB:
            for ns in next_states(s):
                if ns not in ParentB:
                    NewFrontier |= { ns }
                    ParentB[ns]  = s
                    if ns in ParentA:
                        return combinePaths(ns, ParentA, ParentB)
        FrontierB = NewFrontier

Given a `state` and a parent dictionary `Parent`, the function `path_to` returns a path leading to the given `state`.

In [3]:
def path_to(state, Parent):
    p = Parent[state]
    if p == state:
        return [state]
    return path_to(p, Parent) + [state]

The function `combinePath` takes three parameters:
- `state` is a state that has been reached in bidirectional BFS from both `start` and `goal`.
- `ParentA` is the parent dictionary that has been build when searching from `start`.
   If $\texttt{ParentA}[s_1] = s_2$ holds, then either $s_1 = s_2 = \texttt{start}$ or 
   $s_1 \in \texttt{next_states}(s_2)$.
- `ParentB` is the parent dictionary that has been build when searching from `goal`.
   If $\texttt{ParentB}[s_1] = s_2$ holds, then either $s_1 = s_2 = \texttt{goal}$ or
   $s_1 \in \texttt{next_states}(s_2)$.
The function returns a path from `start` to `goal`.

In [10]:
def combinePaths(state, ParentA, ParentB):
        Path1 = path_to(state, ParentA)
        Path2 = path_to(state, ParentB)
        return Path1[:-1] + Path2[::-1] # Path2 is reversed

In [11]:
%run Sliding-Puzzle.ipynb

In [12]:
%load_ext memory_profiler

In [13]:
%%time
%memit Path = search(start, goal, next_states)
print(len(Path)-1)

peak memory: 106.04 MiB, increment: 4.48 MiB
31
CPU times: user 116 ms, sys: 14.2 ms, total: 130 ms
Wall time: 616 ms


In [14]:
animation(Path)

Canvas()

In [15]:
%%time
%memit Path = search(start2, goal2, next_states)
print(len(Path)-1)

peak memory: 912.21 MiB, increment: 805.75 MiB
36
CPU times: user 11.4 s, sys: 290 ms, total: 11.6 s
Wall time: 12.1 s


In [16]:
animation(Path)

Canvas()